In [1]:
import os
import requests
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
options.set_headless(headless=True)
driver = webdriver.Firefox(firefox_options=options)

In [2]:
url = 'https://www.wsj.com/articles/only-a-fraction-of-russian-troll-accounts-have-been-made-public-by-social-med' \
          'ia-giants-1525448056'
filename = '/Users/carlos/Downloads/bstest.html'

In [3]:
driver.get(url)
sleep(10)
driver.find_elements_by_xpath("//*[contains(text(), 'Sign In')]")[0].click()
inputElement = driver.find_element_by_id("username")
inputElement.send_keys(os.environ['WSJ_USER'])
inputElement = driver.find_element_by_id("password")
inputElement.send_keys(os.environ['WSJ_PASS'])
driver.find_elements_by_class_name("solid-button.basic-login-submit")[0].click()

In [4]:
sleep(10)
contents = driver.page_source
soup = BeautifulSoup(contents, 'html.parser')

In [5]:
article = soup.find('article')

# Removing non-content
for id_name in ['ad_and_popular', 'share-target']:
    val = article.find(id=id_name)
    if val is not None:
        val.decompose()

for class_name in ['.comments-count-container', '.author.mobile-scrim', '.byline', '.category']:
    for val in article.select(class_name):
        if val is not None:
            val.decompose()
    
# Format images
for val in article.select('.image-container.responsive-media'):
    if val is not None:
        new_tag = soup.new_tag('img')
        new_tag.attrs['src'] = val.find('img').attrs['src']
        val.replace_with(new_tag)

In [41]:
content = article.prettify("latin-1")
with open(filename, 'wb') as file:
    file.write(content)

In [42]:
driver.quit()